### Prompt Templates

- Prompt templates help to turn raw user information into a format that the LLM can work with.In this case,the raw input is just a message, which we are passing to the LLM.Let's now make that a bit more complicated.First,let's add in a system message with some custom instructions(but still taking messages as input.)Next,we will add in more input besides just the messages.

In [1]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
import os 
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage,SystemMessage
## loading all the enviroments variables
load_dotenv()

# Load the Groq API key from an environment variable
groq_api_key = os.getenv("GROQ_API_KEY")

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant.Answer all the question to the nest of your ability"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)


from langchain_groq import ChatGroq
llm=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
llm


chain=prompt|llm

In [2]:
chain.invoke({"messages":[HumanMessage(content="Hi,My name is Ashu")]})

AIMessage(content="Hello Ashu, it's nice to meet you!\n\nIs there anything I can help you with today? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 37, 'total_tokens': 65, 'completion_time': 0.056925613, 'prompt_time': 0.002881574, 'queue_time': None, 'total_time': 0.059807187}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-751cf7e4-2c4b-4e2d-907f-f8160445b629-0', usage_metadata={'input_tokens': 37, 'output_tokens': 28, 'total_tokens': 65})

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(llm,get_session_history)

In [4]:
config={"configurable":{"session_id":"chat3"}}

In [5]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi,My name is Ashu")],config=config
)
response.content

"Hi Ashu, it's nice to meet you! 👋\n\nWhat can I do for you today? 😊 \n"

In [8]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant.Answer all the question to the nest of your ability in {language}"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt | llm

In [12]:
response=chain.invoke({"messages":[HumanMessage(content="Hi, My name is Ashu")],"language":"Hindi"})
response.content

'नमस्ते अशू! 👋 \n\nमुझे बहुत खुशी है कि आपने मुझे बताया। मैं आपकी मदद करने के लिए तैयार हूँ। क्या आप कोई सवाल पूछना चाहेंगे? 😊\n'

Let's now wrap this more complicated chain in a message history class.This time because there are multiple keys in the inputs,we need to specify the correct key to use to save the chat history.

In [14]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [15]:
config={"configurable":{"session_id":"chat4"}}

In [17]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi, I am ashu")],"language":"hindi" },
    config=config
)
response.content

'नमस्ते अशू!  \n\nआप कैसे हैं? 😊  मुझे आपकी मदद करने में खुशी होगी।  आप क्या पूछना चाहते हैं? \n'